In [23]:
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
import io
import os

from gitdb.fun import chunk_size


def extract_all_from_pdf(pdf_path, output_dir="./extracted_data"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    all_content = {"texts": [], "tables": [], "images": []}

    # 1. PyMuPDF(fitz)로 텍스트와 이미지 추출
    doc = fitz.open(pdf_path)
    # 2. pdfplumber로 테이블 추출 (레이아웃 보존 능력이 뛰어남)
    pdf_plumber = pdfplumber.open(pdf_path)

    for page_num in range(len(doc)):
        print(f"--- {page_num + 1} 페이지 처리 중 ---")

        # [텍스트 추출]
        page = doc.load_page(page_num)
        all_content["texts"].append(page.get_text())

        # [테이블 추출]
        plumber_page = pdf_plumber.pages[page_num]
        table = plumber_page.extract_table()
        if table:
            all_content["tables"].append(table)

        # [이미지 추출]
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            img_filename = f"page{page_num+1}_img{img_index+1}.{image_ext}"
            with open(os.path.join(output_dir, img_filename), "wb") as f:
                f.write(image_bytes)
            all_content["images"].append(img_filename)

    doc.close()
    pdf_plumber.close()
    return all_content

# 실행
content = extract_all_from_pdf("invest/invest.pdf")
print(f"\n추출 완료: 텍스트 {len(content['texts'])}개, 테이블 {len(content['tables'])}개, 이미지 {len(content['images'])}개")

--- 1 페이지 처리 중 ---
--- 2 페이지 처리 중 ---
--- 3 페이지 처리 중 ---

추출 완료: 텍스트 3개, 테이블 0개, 이미지 6개


In [24]:
import os

current_directory = os.getcwd()
fname = "invest.pdf"
fpath = os.path.join(os.path.dirname(current_directory), "ch14/invest/")

print("현재 스크립트의 위치:", current_directory)
print("pdf 위치:",fpath)

현재 스크립트의 위치: /Users/kks/Desktop/Laboratory/jocoding_langchain/ch14
pdf 위치: /Users/kks/Desktop/Laboratory/jocoding_langchain/ch14/invest/


In [25]:
def categorize_elements(raw_pdf_elements: list):
    """
    추출된 요소 리스트를 텍스트와 테이블로 분류합니다.

    :param raw_pdf_elements: 텍스트와 테이블(마크다운)이 섞인 리스트
    :return: (texts 리스트, tables 리스트) 튜플
    """
    texts = []
    tables = []
    for element in raw_pdf_elements:
        # Markdown 테이블 형식인지 간단히 확인 ( '|' 문자로 시작하고 끝나는지)
        # Markdown 테이블은 헤더와 구분선이 필수적으로 포함되므로 이를 기준으로 판단
        if isinstance(element, str) and element.strip().startswith("|") and "---" in element:
            tables.append(element)
        else:
            texts.append(element)
    return texts, tables

In [26]:
raw_pdf_elements = extract_all_from_pdf(fpath+fname)

texts, tables = categorize_elements(raw_pdf_elements)


--- 1 페이지 처리 중 ---
--- 2 페이지 처리 중 ---
--- 3 페이지 처리 중 ---


In [27]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000, chunk_overlap=200)

joined_texts  = " ".join(texts)
texts_2k_token = text_splitter.split_text(joined_texts)

print(len(texts_2k_token))
print(len(texts))

1
3


In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI API 키가 없습니다. 한번 더 확인 부탁드립니다")



In [29]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM



In [31]:
# 텍스트 요소의 요약 생성
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    텍스트 요소 요약 생성
    texts: str 리스트
    tables: str 리스트
    summarize_texts: 텍스트 요약 여부를 나타내는 부울 값
    """

    # Prompt 영어 버전
    # prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    # These summaries will be embedded and used to retrieve the raw text or table elements. \
    # Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """

    # Prompt 한국어 버전
    prompt_text_kor = """당신은 표와 텍스트를 요약하여 검색에 활용할 수 있도록 돕는 도우미입니다. \n
    이 요약본들은 임베딩되어 원본 텍스트나 표 요소를 검색하는 데 사용될 것입니다. \n
    주어진 표나 텍스트의 내용을 검색에 최적화된 간결한 요약으로 작성해 주세요. 요약할 표 또는 텍스트: {element}"""

    prompt = ChatPromptTemplate.from_template(prompt_text_kor)

    # 텍스트 요약 체인 설정
    # 모델: GPT-4o-mini or Llama3.1 모델 사용
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    # llamaModel = OllamaLLM(model="llama3.1:8b")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # 요약 결과를 저장할 빈 리스트 초기화
    text_summaries = []
    table_summaries = []

    # 텍스트가 주어졌고 요약이 요청된 경우 요약을 적용
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        # 요약을 하지 않는 경우 원본 텍스트를 그대로 사용
        text_summaries = texts

    # 테이블이 주어졌을 경우 테이블에 대해 요약 적용
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# 텍스트와 테이블 요약 생성
text_summaries, table_summaries = generate_text_summaries(
    texts_2k_token, tables, summarize_texts=True
)